In [1]:
%pylab inline
import pandas as pd
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


# Testing set: sex determination

In [4]:
df = pd.DataFrame([['Drew', 'No', 'Blue', 'Short', 'Male',],
['Claudia', 'Yes', 'Brown', 'Long', 'Female',],
['Drew', 'No', 'Blue', 'Long', 'Female',],
['Drew', 'No', 'Blue', 'Long', 'Female',],
['Alberto', 'Yes', 'Brown', 'Short', 'Male',],
['Karin', 'No', 'Blue', 'Long', 'Female',],
['Nina', 'Yes', 'Brown', 'Short', 'Female',],
['Sergio', 'Yes', 'Blue', 'Long', 'Male',]])
df.columns = ['Name', 'isOver170', 'isEyeColorBrown', 'isHairLong', 'isFemale']

df.loc[df['isEyeColorBrown']=='Brown', ['isEyeColorBrown']] = 1
df.loc[df['isEyeColorBrown']=='Blue', ['isEyeColorBrown']] = 0

df.loc[df['isOver170']=='Yes', ['isOver170']] = 1
df.loc[df['isOver170']=='No', ['isOver170']] = 0

df.loc[df['isHairLong']=='Long', ['isHairLong']] = 1
df.loc[df['isHairLong']=='Short', ['isHairLong']] = 0

df.loc[df['isFemale'] == 'Female', ['isFemale']]= 1
df.loc[df['isFemale'] == 'Male', ['isFemale']]= 0

from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
data = vec.fit_transform(df.to_dict('records')).toarray()
observations, labels = data[:,:-1],data[:,-1]
training_data, testing_data = observations[:-1], observations[-1]
training_dataset_compiled = [training_data, labels[:-1]]
testing_dataset_compiled = [testing_data, labels[-1]]

My naive bayes implementation based on 598 lecture notes:

In [26]:
def naive_bayes(X,Y, gamma=1):
    unique_labels = np.unique(Y) 

    data_array = np.array(X) # the data in array format


    n = data_array.shape[0] # number of observations we have
    m = data_array.shape[1] # number of features
    k = len(unique_labels) # number of classes
    t = 2 #assuming x_i are boolean variables

    labels_array = np.array(Y)
    
    theta = (gamma/float(gamma*t)) * np.ones((m,t,k))
    pie = np.zeros(k) # filling in default values as the smoothening factor
    
    unique_feature_labels = [None]*m # array to hold the unique values of each feature

    for label_index, label in enumerate(unique_labels):
        print 'y_k=',label
        matching_indicies = labels_array == label
        pie[label_index] = (np.sum(matching_indicies)+gamma)/(float(n)+gamma*k)
        print 'pie[k]=',pie[label_index]
        # obtain all the data that matches our labels.
        data_for_label = data_array[matching_indicies]

        # transpose the matrix because we want to now calculate the counts of the features.
        feature_observations = data_for_label.T

        for feature_index, feature_observation in enumerate(feature_observations):

            unique_feature_labels[feature_index] = list(sorted(np.unique(feature_observation)))

            for count in Counter(feature_observation).items():
                t_instance, N_condition = count[0], count[1] # value, frequency 

                theta[feature_index, t_instance, label_index] = (N_condition + gamma)/float(np.sum(matching_indicies) + gamma * t)
                print 'theta(j=',feature_index,', t=',t_instance,'k=',label_index,') = ',theta[feature_index, t_instance, label_index]
            #endfor
        #endfor
    #endfor
    
    def predictor(X):
        to_return = []
        for to_classify in X:
            print 'to_classify:',to_classify
            results = []
            for label in unique_labels:
                p = pie[label]
                print 'P(y=',label,')=',pie[label]
                for j, val in zip(range(m), to_classify):
                    print 'P(X_',j,'=',val,' | y=',label,')=', theta[j,val,label]
                    p = p * theta[j,val,label]
                    print '\n'
                #endfor
                results.append((label, p))
            print 'Ps: ',results
            print 'Prediction:', max(results,key=lambda l: l[1])
            #endfor
            to_return.append( max(results, key=lambda l: l[1]) )
        #endfor
        return to_return
    return predictor


In [27]:
pred = naive_bayes(*training_dataset_compiled)

y_k= 0.0
pie[k]= 0.555555555556
theta(j= 0 , t= 0.0 k= 0 ) =  0.833333333333
theta(j= 1 , t= 0.0 k= 0 ) =  0.833333333333
theta(j= 2 , t= 0.0 k= 0 ) =  0.333333333333
theta(j= 2 , t= 1.0 k= 0 ) =  0.666666666667
theta(j= 3 , t= 0.0 k= 0 ) =  0.666666666667
theta(j= 3 , t= 1.0 k= 0 ) =  0.333333333333
theta(j= 4 , t= 0.0 k= 0 ) =  0.833333333333
theta(j= 5 , t= 0.0 k= 0 ) =  0.833333333333
theta(j= 6 , t= 0.0 k= 0 ) =  0.833333333333
theta(j= 7 , t= 0.0 k= 0 ) =  0.333333333333
theta(j= 7 , t= 1.0 k= 0 ) =  0.666666666667
theta(j= 8 , t= 0.0 k= 0 ) =  0.333333333333
theta(j= 8 , t= 1.0 k= 0 ) =  0.666666666667
y_k= 1.0
pie[k]= 0.444444444444
theta(j= 0 , t= 0.0 k= 1 ) =  0.6
theta(j= 0 , t= 1.0 k= 1 ) =  0.4
theta(j= 1 , t= 0.0 k= 1 ) =  0.6
theta(j= 1 , t= 1.0 k= 1 ) =  0.4
theta(j= 2 , t= 0.0 k= 1 ) =  0.8
theta(j= 3 , t= 0.0 k= 1 ) =  0.8
theta(j= 4 , t= 0.0 k= 1 ) =  0.6
theta(j= 4 , t= 1.0 k= 1 ) =  0.4
theta(j= 5 , t= 0.0 k= 1 ) =  0.8
theta(j= 6 , t= 1.0 k= 1 ) =  0.8
theta(j= 7 

/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:37: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [28]:
pred([testing_dataset_compiled[0]])

to_classify: [ 0.  0.  0.  0.  0.  1.  0.  0.  1.]
P(y= 0.0 )= 0.555555555556
P(X_ 0 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 1 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 2 = 0.0  | y= 0.0 )= 0.333333333333


P(X_ 3 = 0.0  | y= 0.0 )= 0.666666666667


P(X_ 4 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 5 = 1.0  | y= 0.0 )= 0.5


P(X_ 6 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 7 = 0.0  | y= 0.0 )= 0.333333333333


P(X_ 8 = 1.0  | y= 0.0 )= 0.666666666667


P(y= 1.0 )= 0.444444444444
P(X_ 0 = 0.0  | y= 1.0 )= 0.6


P(X_ 1 = 0.0  | y= 1.0 )= 0.6


P(X_ 2 = 0.0  | y= 1.0 )= 0.8


P(X_ 3 = 0.0  | y= 1.0 )= 0.8


P(X_ 4 = 0.0  | y= 1.0 )= 0.6


P(X_ 5 = 1.0  | y= 1.0 )= 0.5


P(X_ 6 = 0.0  | y= 1.0 )= 0.5


P(X_ 7 = 0.0  | y= 1.0 )= 0.4


P(X_ 8 = 1.0  | y= 1.0 )= 0.4


Ps:  [(0.0, 0.0066152686751680791), (1.0, 0.0024576000000000003)]
Prediction: (0.0, 0.0066152686751680791)


/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:49: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:50: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:52: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:53: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[(0.0, 0.0066152686751680791)]

In [29]:
out = pred(training_dataset_compiled[0])

to_classify: [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
P(y= 0.0 )= 0.555555555556
P(X_ 0 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 1 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 2 = 1.0  | y= 0.0 )= 0.666666666667


P(X_ 3 = 0.0  | y= 0.0 )= 0.666666666667


P(X_ 4 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 5 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 6 = 0.0  | y= 0.0 )= 0.833333333333


P(X_ 7 = 0.0  | y= 0.0 )= 0.333333333333


P(X_ 8 = 0.0  | y= 0.0 )= 0.333333333333


P(y= 1.0 )= 0.444444444444
P(X_ 0 = 0.0  | y= 1.0 )= 0.6


P(X_ 1 = 0.0  | y= 1.0 )= 0.6


P(X_ 2 = 1.0  | y= 1.0 )= 0.5


P(X_ 3 = 0.0  | y= 1.0 )= 0.8


P(X_ 4 = 0.0  | y= 1.0 )= 0.6


P(X_ 5 = 0.0  | y= 1.0 )= 0.8


P(X_ 6 = 0.0  | y= 1.0 )= 0.5


P(X_ 7 = 0.0  | y= 1.0 )= 0.4


P(X_ 8 = 0.0  | y= 1.0 )= 0.6


Ps:  [(0.0, 0.011025447791946801), (1.0, 0.0036863999999999998)]
Prediction: (0.0, 0.011025447791946801)
to_classify: [ 0.  1.  0.  0.  0.  0.  1.  1.  1.]
P(y= 0.0 )= 0.555555555556
P(X_ 0 = 0.0  | y= 0.0 )= 0.833333333

/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:49: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:50: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:52: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/zaf/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:53: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [30]:
out

[(0.0, 0.011025447791946801),
 (0.0, 0.0079383224102016967),
 (0.0, 0.044101791167787202),
 (0.0, 0.044101791167787202),
 (1.0, 0.0062914560000000034),
 (0.0, 0.011025447791946801),
 (1.0, 0.0094371840000000012)]

In [37]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb_trained = nb.fit(*training_dataset_compiled)
nb_trained.predict_proba(training_dataset_compiled[0])

array([[ 0.95224214,  0.04775786],
       [ 0.13008849,  0.86991151],
       [ 0.987617  ,  0.012383  ],
       [ 0.987617  ,  0.012383  ],
       [ 0.03603824,  0.96396176],
       [ 0.95224214,  0.04775786],
       [ 0.04748062,  0.95251938]])

In [39]:
np.exp(nb_trained.feature_log_prob_)

array([[ 0.16666667,  0.16666667,  0.66666667,  0.33333333,  0.16666667,
         0.16666667,  0.16666667,  0.66666667,  0.66666667],
       [ 0.4       ,  0.4       ,  0.2       ,  0.2       ,  0.4       ,
         0.2       ,  0.8       ,  0.6       ,  0.4       ]])

In [40]:
np.exp(nb_trained.class_log_prior_)

array([ 0.57142857,  0.42857143])